# Cortisol Analysis – All Log Types

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_log_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import (
    create_unique_night_id
)

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette}
theme_kwargs_grid = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 0.8}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
#result_path = base_path.joinpath("results")
#plot_path = result_path.joinpath("plots")
#stats_path = result_path.joinpath("statistics")

#bp.utils.file_handling.mkdirs([result_path, plot_path, stats_path])

## Load Data

### Cortisol Samples

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_log_types.csv")
cort_samples = load_cortisol_samples_log_times(cort_path)

cort_samples.head()

### Cortisol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_log_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

In [ ]:
conditions = ["Spontaneous", "Known Alarm", "Unknown Alarm"]
# Don't consider IMU and IMU_App because the cortisol features are the same as 
# Selfreport and App since only the wake onset differs, but not the sampling times
log_types = ["Naive", "Selfreport", "App"]
log_types_full = ["Naive", "Selfreport", "App", "Sensor_Selfreport", "Sensor_App"]
delay_groups = ["None", "Short", "Moderate", "High"]

## Data Selection

### Cortisol Samples

In [ ]:
cort_samples = multi_xs(cort_samples, log_types_full, level="log_type")
cort_samples.head()

### Cortisol Features

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = multi_xs(cort_features_analysis, ["auc_g", "auc_i"], level="saliva_feature")
cort_features_analysis = multi_xs(cort_features_analysis, log_types, level="log_type")
cort_features_analysis.head()

## Saliva Samples – CAR Plot

In [ ]:
car = bp.protocols.CAR()
car.add_saliva_data(
    saliva_data=cort_samples.drop("Naive", level="log_type"),
    saliva_type="cortisol",
    sample_times=[0, 15, 30, 45, 60]
)

In [ ]:
fig, ax = plt.subplots()

car.car_saliva_plot(
    saliva_type="cortisol", 
    hue="log_type", 
    style="log_type",
    #linestyle=["-", "--", "-.", ":"],
    #marker=["o", "P", "v", "s"],
    ax=ax, 
    hue_order=log_types_full[1:], 
    x_offset=0.0
)
fig.tight_layout()

## Saliva Features – Statistical Analysis

### Log Type

In [ ]:
data_analysis = cort_features_analysis.copy()

pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"), 
        ("prep", "equal_var"), 
        ("test", "rm_anova"), 
        ("posthoc", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol", 
        "within": "log_type", 
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": False}
    },
)

pipeline.apply(data_analysis)
pipeline.display_results()

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", 
    stats_effect_type="within", 
    plot_type="single",
    subplots=True,
    x="log_type",
    features=["auc_g", "auc_i"]
)

fig, axs = plt.subplots(ncols=2)

for (feature, data), ax in zip(cort_features_analysis.groupby("saliva_feature"), axs):
    bp.protocols.plotting.saliva_feature_boxplot(
        data=data, 
        x="log_type",
        saliva_type="cortisol",
        feature=feature,
        order=log_types,
        stats_kwargs={"box_pairs": box_pairs[feature], "pvalues": pvalues[feature], "verbose": 0},
        ax=ax
    )
    ax.set_title(feature)

fig.tight_layout()

### Interaction Condition x Log Type

In [ ]:
data_analysis = cort_features_analysis.copy()

pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"), 
        ("prep", "equal_var"), 
        ("test", "mixed_anova"), 
        ("posthoc", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol", 
        "within": "log_type", 
        "between": "condition",
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": None}
    },
)

pipeline.apply(data_analysis)
pipeline.display_results(prep=False, posthoc=False)

### Paired Plots

#### $AUC_G$

In [ ]:
fig, axs = plt.subplots(ncols=3, sharey=True)

data_plot = cort_features_analysis.xs("auc_g", level="saliva_feature")

for log_type, ax in zip(log_types[::-1], axs):
    order = log_types.copy()
    order.remove(log_type)
    
    data = data_plot.reindex(order, level="log_type")
    
    pg.plot_paired(
        data=data.reset_index(), 
        dv="cortisol", 
        within="log_type", 
        order=order,
        subject="night_id", 
        #boxplot_in_front=True, 
        pointplot_kwargs={"alpha": 0.5},
        ax=ax
    )

fig.suptitle("$AUC_G$")
fig.tight_layout()

#### $AUC_I$

In [ ]:
fig, axs = plt.subplots(ncols=3, sharey=True)

data_plot = cort_features_analysis.xs("auc_i", level="saliva_feature")

for log_type, ax in zip(log_types[::-1], axs):
    order = log_types.copy()
    order.remove(log_type)
    
    data = data_plot.reindex(order, level="log_type")
    
    pg.plot_paired(
        data=data.reset_index(), 
        dv="cortisol", 
        within="log_type", 
        order=order,
        subject="night_id", 
        #boxplot_in_front=True, 
        pointplot_kwargs={"alpha": 0.5},
        ax=ax
    )

fig.suptitle("$AUC_I$")
fig.tight_layout()

### Increase vs. S2 Time Delay

(Kudielka et al. 2003):  
"We found that the larger the time deviation for sample 2 (+30 min), the smaller the observed awakening cortisol increase. If subjects delay sample 2, they obviously miss the peak, and the resulting awakening increase turns out to be smaller."

#### Data Preparation

Get time deviation for S2 (+30 min)

In [ ]:
s2_delay = cort_samples.xs("S2", level="sample")[["time_diff_to_naive_min"]].dropna()

# drop time deviation outlier, i.e., samples that are be closer to S3 or to S1 than S2 (|delay| >= 7.5 min)
drop_mask = s2_delay["time_diff_to_naive_min"].abs() >= 7.5
drop_mask = drop_mask[drop_mask]
s2_delay = s2_delay.drop(drop_mask.index)
s2_delay.columns = ["s2_delay"]

s2_delay.head()

Get cortisol increase between S0 and S2

In [ ]:
cort_inc = cort_samples[["cortisol"]]
cort_inc = (cort_inc.xs("S2", level=-1) - cort_inc.xs("S0", level=-1))
cort_inc = cort_inc.join(s2_delay).dropna()
cort_inc.head()

#### Linear Regression

In [ ]:
data_grp = cort_inc.groupby("log_type")

data_result = {}

for log_type in ["Selfreport", "App", "Sensor_App"]:
    data_reg = data_grp.get_group(log_type)
    reg = pg.regression.linear_regression(
        X=data_reg["s2_delay"],
        y=data_reg["cortisol"],
    )
    data_result[log_type] = reg
    
pd.concat(data_result)

#### Regression Plot

In [ ]:
fig,  ax = plt.subplots()

for log_type in ["Selfreport", "App", "Sensor_App"]:
    sns.regplot(
        data=cort_inc.xs(log_type, level="log_type").reset_index(), 
        x="s2_delay", 
        y="cortisol", 
        ax=ax, 
        label=log_type
    )
ax.legend()
fig.tight_layout()